In [ ]:
import pandas as pd
import numpy as np
import itertools
import random
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, StandardScaler

In [ ]:
data_1 = pd.read_csv('Trip List Edited 12.1.2022-2.1.2023.csv', parse_dates=['Date of Service'], index_col = 'Date of Service')
data_2 = pd.read_csv('Trip List Edited 2.1.2023-4.1.2023.csv', parse_dates=['Date of Service'], index_col = 'Date of Service')
data_3 = pd.read_csv('Trip List Edited 4.1.2023-6.1.2023.csv', parse_dates=['Date of Service'], index_col = 'Date of Service')
data_4 = pd.read_csv('Trip List Edited 8.1.2023-12.21-2023.csv', parse_dates=['Date of Service'], index_col = 'Date of Service')
data_current_cycle = pd.read_csv('Trip List Edited Current.csv', parse_dates=['Date of Service'], index_col = 'Date of Service')

#data = pd.concat([data_1, data_2, data_3, data_4])
data = pd.concat([data_1, data_2, data_3, data_4, data_current_cycle])

data = data.dropna()

# with pd.option_context('display.max_rows', None,):
#     print(data)

data

In [ ]:
#data['Date of Service'] = pd.to_datetime(data['Date of Service'])
#data.set_index('Date of Service')
#data.sort_index()
data['Location'] = data['Location'].str.strip()

# Filtering for Jacksonville location
data_location = data[data['Location'] == 'PONTE VEDRA BEACH']

# Parameters for the model and forecast
# n_test = 14  # Number of days for testing

data_location #Now has only Jacksonville locations

In [ ]:
#Encode Data
label_encoders = {}
for column in ['Call Type', 'Priority', 'Nature of Call']:
    encoder = LabelEncoder()
    data_location[column] = encoder.fit_transform(data_location[column])
    label_encoders[column] = encoder

#grouped_data = data.groupby(['Date of Service','Location', 'Unit', 'Call Type', 'Priority', 'Nature of Call']).size().reset_index(name='Daily Count')
#grouped_data = grouped_data.sort_values(by='Date of Service')

#Scaling exogenous values
features_to_normalize = ['Call Type', 'Priority', 'Nature of Call']
scaler = MinMaxScaler() #0 Values to represent Unit not being there for that day. 
data_location[features_to_normalize] = scaler.fit_transform(data_location[features_to_normalize]) 
data_location
#grouped_data[features_to_normalize] = scaler.fit_transform(grouped_data[features_to_normalize])
#grouped_data

The cell below uses a grid search to pick the best parameters for SARIMAX

In [ ]:
# Define the parameter space for SARIMAX
p = d = q = range(0, 3)  # Example ranges for ARIMA orders
P = D = Q = range(0, 3)
s = [7, 14, 30]  #Testing for Weekly, Bi-Weely, and Monthly Seasonality

# Random Grid Search across all units
parameters = list(itertools.product(p, d, q, P, D, Q, s))
pdq_combinations = random.sample(parameters, min(len(parameters), 5))  # Randomly sample combinations

In [ ]:
pdq_combinations

In [ ]:
#pdq_combinations = [(1, 0, 2, 2, 1, 1, 7), ] #Testing specific parameters. 
pdq_combinations.extend(((0, 0, 0, 1, 1, 1, 14),))
pdq_combinations

Section: Used for testing Code

In [ ]:
# # Apply best parameters for each unit to test
# unit_IDs = sorted(data_location['Unit'].unique())

# #Splitting training and testing by date
# split_date = pd.Timestamp('2023-11-30')

# start_date = data_location.index.min()
# end_date = data_location.index.max()
# date_range = pd.date_range(start= start_date, end= end_date, freq = 'D')
# exog_data = data_location[['Call Type', 'Priority', 'Nature of Call']].groupby(data_location.index).mean().reindex(
#                                                                                                             date_range, 
#                                                                                                             fill_value = 0)

# for unit_ID in unit_IDs:
#     print(f"Processing Medic Unit: {unit_ID}")

#     unit_data_grouped = data_location[data_location['Unit'] == unit_ID].resample('D').count().reindex(date_range, 
#                                                                                                       fill_value = 0)
#     unit_data_grouped = unit_data_grouped.drop('Location', axis = 1)
#     #print(unit_data_grouped['Unit'][0])

#     for i in range(len(unit_data_grouped)):
#         if unit_data_grouped['Unit'][i] > 0:
#             unit_data_grouped['Call Type'][i] = exog_data['Call Type'][i]
#             unit_data_grouped['Priority'][i] = exog_data['Priority'][i]
#             unit_data_grouped['Nature of Call'][i] = exog_data['Nature of Call'][i]
            
#             data_unit = unit_data_grouped['Unit']
#             exog_unit = unit_data_grouped[['Call Type', 'Priority', 'Nature of Call']]
            
#     #print(exog_unit)
    
#     #train_end = pd.to_datetime("11/30/2023")
#     train = data_unit[data_unit.index < split_date]
#     test = data_unit[data_unit.index >= split_date]

#     train_exog = exog_unit[exog_unit.index < split_date]
#     test_exog = exog_unit[exog_unit.index >= split_date]
    
#     print(data_unit)

Section: Code Testing - End

In [ ]:
def sarimax_grid_search(data_location, pdq_combinations): #Aggregated order and seasonal order. 
#     best_aic = float("inf")
#     best_configuration = None
#     best_model = None
    
    # Apply best parameters for each unit to test
    unit_IDs = sorted(data_location['Unit'].unique())
    
    #Used for training/test split further below. 
    split_date = pd.Timestamp('2023-11-30')
    
    #Preparing exogenous values
    start_date = data_location.index.min()
    end_date = data_location.index.max()
    date_range = pd.date_range(start= start_date, end= end_date, freq = 'D')
    exog_data = data_location[['Call Type', 'Priority', 'Nature of Call']].groupby(data_location.index).mean().reindex(
                                                                                                                date_range, 
                                                                                                                fill_value = 0)
    # Initialize a dictionary to store best_configuration
    unit_results={}
    
    for unit_ID in unit_IDs:
        best_aic = float("inf")
        best_configuration = None
        best_model = None
        
        print(f"Processing Medic Unit: {unit_ID}")
        
        #Using date_range from preparing exogenous values
        unit_data_grouped = data_location[data_location['Unit'] == unit_ID].resample('D').count().reindex(date_range, 
                                                                                                          fill_value = 0)
        unit_data_grouped = unit_data_grouped.drop('Location', axis = 1)
        #print(unit_data_grouped['Unit'][0])

        for i in range(len(unit_data_grouped)):
            if unit_data_grouped['Unit'][i] > 0:
                unit_data_grouped['Call Type'][i] = exog_data['Call Type'][i]
                unit_data_grouped['Priority'][i] = exog_data['Priority'][i]
                unit_data_grouped['Nature of Call'][i] = exog_data['Nature of Call'][i]
        
        data_unit = unit_data_grouped['Unit']
        exog_unit = unit_data_grouped[['Call Type', 'Priority', 'Nature of Call']]
        
        #train_end = pd.to_datetime("11/30/2023")
        train = data_unit[data_unit.index < split_date]
        test = data_unit[data_unit.index >= split_date]
        
        train_exog = exog_unit[exog_unit.index < split_date]
        test_exog = exog_unit[exog_unit.index >= split_date]
        
        #For lesser computers.
        for pdq in pdq_combinations:
            try:
                model = SARIMAX(train, order=pdq[:3], seasonal_order=pdq[3:], exog = train_exog, 
                                enforce_stationarity=False, enforce_invertibility=False)
                model_fit = model.fit(disp=False)

                if model_fit.aic < best_aic: 
                    best_aic = model_fit.aic
                    best_configuration = pdq
                    best_model = model_fit

            except Exception as e: 
                print(f'Exception: {e}')
                continue
        
        #Storing best_configuration
        unit_results[unit_ID] = {'best_configuration' : best_configuration}
        
        if best_configuration is None:
            raise ValueError("No suitable model parameters found for aggregate data")
            
        #Predicting the rest of the forecast to compare to test data. Steps has to be the remaining days to predict. 
        predictions = best_model.forecast(steps=len(test), exog = test_exog)
#         predictions = best_model.get_forecast(steps=len(test), exog = test_exog)
#         forecast_values = predictions.predict_mean

        # Performance Metrics
        mse = mean_squared_error(test, predictions)
        mae = mean_absolute_error(test, predictions)
        rmse = np.sqrt(mse)
        print(f'Unit {unit_ID} - Best SARIMAX: {best_configuration} - MSE: {mse}, MAE: {mae}, RMSE: {rmse}')

        # Plotting Forecast vs Actual
        plt.figure(figsize=(10, 6))
        plt.plot(test, label='Actual')
        plt.plot(predictions, label='Forecast', color='red')
        plt.title(f'SARIMAX Model Forecast vs Actual for Unit {unit_ID}')
        plt.xlabel('Date')
        plt.ylabel('Unit')
        plt.legend()
        plt.show()
            
    return best_aic, best_configuration, best_model, unit_results

In [ ]:
# #Running Grid Search
# #SARIMAX can internally difference with value (d). We manually difference (d=1) just to check stationarity. 
# #Now we will let SARIMAX integrate difference by setting d=1 manually on the ORIGINAL dataset. 

#ts_data = df['Actual Trips']
#ts_data = train['Unit']

ts_data = data_location

# exog_train = train[['Call Type', 'Priority', 'Nature of Call']]
# exog_test = test[['Call Type', 'Priority', 'Nature of Call']]

# #For lesser PC
best_aic, best_configuration, best_model, unit_results = sarimax_grid_search(ts_data, pdq_combinations)

#print(f"Best SARIMAX parameters: {best_configuration} with AIC: {best_aic}")

In [ ]:
best_configuration #You can interupt cell above to get the current best_params value without having to finish all the calculations. 

In [ ]:
best_config = unit_results['MEDIC 203'].values()
results = list(best_config)
results[0]

Everything below is to generate future values. (Needs to be reworked.) 

In [ ]:
#The best parameters for all Units combined; Unit MEDIC 1 usually has highest RSME so minimizing this
    #Best SARIMAX: (0, 0, 0, 2, 0, 1, 7) - MSE: 3.477645430848429, MAE: 1.3565603776833444, RMSE: 1.8648446130571

unique_locations = data['Location'].unique()
print("Unique Locations:", unique_locations)

In [ ]:
# Correct the filtering condition based on the actual string in your dataset
data_location = data[data['Location'] == 'PONTE VEDRA BEACH']

# Get unique 'Medic Units' 
unit_IDs = data_location['Unit'].unique()
unit_IDs = sorted(unit_IDs)
print("Unique Units in Ponte Vedra Beach:", unit_IDs)

In [ ]:
print("Unique Vehicle IDs in Ponte Vedra Beach:", unit_IDs)

#Preparing exogenous values
#Using date_range from preparing exogenous values
start_date = data_location.index.min()
end_date = data_location.index.max()
date_range = pd.date_range(start= start_date, end= end_date, freq = 'D')

models = {} #Fitting model for all units individually.
for unit_ID in unit_IDs:
    print(f"Processing Call Type: {unit_ID}")

    # Aggregate data by day for each 'Unit'
    ts = data_location[data_location['Unit'] == unit_ID].resample('D').count().reindex(date_range, fill_value = 0)
    ts = ts.drop('Location', axis = 1)
    ts_data = ts['Unit']
    
    #print(f"Sample Data for {unit_ID}:\n", ts.head())  # Print sample data
    #Retrieve best_configuration for current unit
    results = unit_results[unit_ID].values()
    results = list(results)
    best_configuration = results[0]
    
    # Check if time series is not empty
    if not ts.empty:
        model = SARIMAX(ts_data, order=best_configuration[:3], seasonal_order=best_configuration[3:], #exog = train_exog, #Implementing later
                       enforce_stationarity=False, enforce_invertibility=False) 
        models[unit_ID] = model.fit(disp=True)
    else:
        print(f"No data available for Call Type: {unit_ID}")

In [ ]:
forecasts = {}
steps = 14
for unit_ID, model in models.items():
    forecasts[unit_ID] = model.forecast(steps)  # Forecasting for the next day

In [ ]:
from datetime import timedelta

# Determine the date for the forecast
# Assuming you are forecasting for the next day after the last date in your data
last_date = data.index.max()
start_forecast_date = last_date + timedelta(days=1)

# Initialize a dictionary to store forecasts for each date
date_forecasts = {start_forecast_date + timedelta(days=i): [] for i in range(steps)}

# Generate and store forecasts for each call type and each date
for unit_ID in unit_IDs:
    model = models.get(unit_ID)
    if model:
        forecast_values = model.forecast(steps=steps)
        forecast_values = abs(forecast_values)
        for i in range(steps):
            forecast_date = start_forecast_date + timedelta(days=i)
            # Append the forecast string for each call type and date
            #date_forecasts[forecast_date].append(f"{forecast_values[i]:.0f} calls of type '{vehicle_ID}'")
            #date_forecasts[forecast_date].append(f"{vehicle_ID}: {forecast_values[i]:.0f}")
            date_forecasts[forecast_date].append(f"{unit_ID}: {forecast_values[i]:.0f}")
    else:
        # Handle case where there is no model for a call type
        for date in date_forecasts:
            date_forecasts[date].append(f"0 calls for unit '{unit_ID}'")

# Print the combined forecast message for each date
for date, forecasts in date_forecasts.items():
    forecast_message = " \n".join(forecasts)
    print(f"There will be: \n{forecast_message}\nin the location of Ponte Vedra Beach{date.strftime('%Y-%m-%d')}")

In [ ]:
#Creating an empty DataFrame with specified columns
#forecast_results_df = pd.DataFrame(columns=['Date of Service', 'Call Type & Volume', 'Location'])

# Initialize a list to store forecast data for DataFrame
forecast_data = []

#Generate forecast data suitable for DataFrame
for date, forecasts in date_forecasts.items():
    #forecast_message = " and ".join(forecasts)
    forecast_message = ' , '.join(forecasts)
    forecast_data.append({
        "Date of Service": date.strftime('%A: %m-%d-%Y'),
        "Unit": forecast_message,
        "Location": "Ponte Vedra Beach"  # or dynamically set location if needed
    })
    
# Create a DataFrame
forecast_df = pd.DataFrame(forecast_data)
forecast_df

In [ ]:
# Define the output file path
output_file_path = 'Model Forecast Results.csv'

# Saving the DataFrame to a CSV file
try:
    forecast_df.to_csv(output_file_path, mode='a', index=False, header=False)
    success = True
except Exception as e:
    success = False
    error_message = str(e)

# Returning the path if successful, else the error message
output_file_path if success else error_message